In [1]:
import numpy as np
import pandas as pd

Я работаю в гугл колабе и беру данные с гугл диска. При необходимости поменяйте два блока внизу на нужный вам способ хранения и получения данных.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/Recsys_data1.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
def analysis(data: pd.DataFrame):
    data = data.drop(['oaid_hash', 'banner_id0', 'banner_id1', 'rate0', 'rate1', 'g0', 'g1', 'coeff_sum0',  'coeff_sum1'], axis=1) #remove unnecessary cols (according to the task)
    print(data.head)
    print("Data contains null is ", data.isnull().values.any()) #check if contains null
    print(len(df[df['impressions'] == 0])) #check if impressions contains 0
    #count how many data on each day we have
    cur = data.copy()
    cur['date_time'] = cur['date_time'].apply(lambda x: x[:10])
    cur = cur.groupby(['date_time']).size()
    print(cur)
    return data

In [6]:
df = analysis(df)

<bound method NDFrame.head of                            date_time  zone_id  banner_id  campaign_clicks  \
0         2021-09-27 00:01:30.000000        0          0                0   
1         2021-09-26 22:54:49.000000        1          1                0   
2         2021-09-26 23:57:20.000000        2          2                3   
3         2021-09-27 00:04:30.000000        3          3                0   
4         2021-09-27 00:06:21.000000        4          4                0   
...                              ...      ...        ...              ...   
15821467  2021-10-02 15:51:35.000000      146        530                0   
15821468  2021-09-27 22:03:14.000000       12         22                0   
15821469  2021-10-02 17:41:10.000000       12       1236                0   
15821470  2021-09-29 00:39:32.000000      967         21                0   
15821471  2021-09-28 07:00:18.000000       19        635                0   

          os_id  country_id  impressions  cli

Выводы из анализа данных:
1. Есть ненужная числовая зависимость в колонках zone_id, banner_id, os_id, country_id, значит в обработке будем использовать например OneHotEncoding.

2. В данных нет null.
3. Все impressions равны 1 (что логично), так что уберем этот столбец.

4. Также есть день 2021-09-01, для которого есть всего одна запись и он далеко от остальных. Видимо он попал в данные случайно, его надо удалить.

5. Последний день (который будем брать в тест) 2021-10-02.

Ровно этим и будем заниматься в функции feature_engineering. Правда в OneHotEncoding отправим все данные, потому что, они так сожмутся. А без сжатия трудно закодировать даже один столбец.



In [7]:
from sklearn.preprocessing import OneHotEncoder

#feature engineering according to the analysis and spliting into train and test
def feature_engineering(data: pd.DataFrame) -> pd.DataFrame:
    data = data.drop(['impressions'], axis=1)
    data = data[data['date_time'] > '2021-09-20']
    enc = OneHotEncoder(handle_unknown='ignore')
    enc = enc.fit(data.drop(['clicks', 'date_time'], axis=1))
    cur = data[data['date_time'] < '2021-10-02']
    x_train = cur.drop(['clicks', 'date_time'], axis=1)
    x_train = enc.transform(x_train)
    y_train = cur['clicks']
    cur = data[data['date_time'] >= '2021-10-02']
    x_test = cur.drop(['clicks', 'date_time'], axis=1)
    x_test = enc.transform(x_test)
    y_test = cur['clicks']
    return x_train, y_train, x_test, y_test

In [8]:
x_train, y_train, x_test, y_test = feature_engineering(df)
df = []

Дальше создаем макет линейной модели. В качестве оптимизатора берем liblinear. 

In [9]:
from sklearn.linear_model import LogisticRegression

def create_model(fit_intercept=False, solver='liblinear', C=0.5, penalty='l2'):
    model = LogisticRegression(solver='liblinear', C=C, penalty=penalty, fit_intercept=fit_intercept)
    return model

Делаем тесты с l2 регуляризацией.

In [ ]:
from sklearn.model_selection import cross_validate
c_arr_l2 = [0.001, 0.01, 0.1, 1.0, 10.0]

for C in c_arr_l2:
    model = create_model(C=C)
    scores = cross_validate(model, x_train, y_train, scoring=['neg_log_loss', 'roc_auc'], cv=4)
    print(C)
    print('mean_test_neg_log_loss', np.mean(scores['test_neg_log_loss']))
    print('test_roc_auc', np.mean(scores['test_roc_auc']))

0.001
mean_test_neg_log_loss -0.11093903850984717
test_roc_auc 0.7059417789326434
0.01
mean_test_neg_log_loss -0.109815321105411
test_roc_auc 0.7178021331020474
0.1
mean_test_neg_log_loss -0.11000979333761407
test_roc_auc 0.7164108310476427
1.0
mean_test_neg_log_loss -0.11025877812638318
test_roc_auc 0.7153793959727327
10.0
mean_test_neg_log_loss -0.11043268421479822
test_roc_auc 0.7149254488556916


Также я проводила эксперименты с l1 регуляризацией. Они работали в несколько раз дольше, но не давали значительного прироста метрик. Еще были проведены эксперементы с наличием свободного коэффициента (fit_interceipt=True), но это также не оказало влияния на результат. 

В итоге как оптимальный из рассмотренных вариантов берем l2 регуляризацию с коэффициентом 0.01.

In [42]:
from sklearn.metrics import roc_auc_score, log_loss
best_C = 0.01
model = create_model(C=best_C)
model = model.fit(x_train, y_train)
y_pred = model.predict(x_test)
auc_metric = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
log_loss_metric = log_loss(y_test, model.predict_proba(x_test))
print('Auc: {:.3f}, log_loss: {:.3f}'.format(auc_metric, log_loss_metric))

Auc: 0.779, log_loss: 0.134


Теперь сравним с бейзлайном.

In [43]:
y_pred_base = np.full(y_pred.shape, np.mean(y_train))
auc_metric_base = roc_auc_score(y_test, y_pred_base)
log_loss_metric_base = log_loss(y_test, y_pred_base)
print('Auc base: {:.3f}, log_loss base: {:.3f}'.format(auc_metric_base, log_loss_metric_base))

Auc base: 0.500, log_loss base: 0.155


Наша модель по метрикам превзошла бейзлайн. Ура.